In [ ]:
import yfinance as yf
import pandas as pd

# Coletar histórico do dólar (USDBRL=X) e da PETR4
tickers = ["USDBRL=X", "PETR4.SA"]

# Baixar dados ajustados
data = yf.download(tickers, start="2023-01-01", end="2024-12-31")["Close"]

# Resetar index e renomear colunas
df = data.reset_index()
df.columns = ["Data", "Dólar", "PETR4"]

# Interpolar dados faltantes do dólar primeiro
df["Dólar"] = df["Dólar"].interpolate()

# Criar retornos percentuais
df["Retorno_Dólar_%"] = df["Dólar"].pct_change(fill_method=None) * 100
df["Retorno_PETR4_%"] = df["PETR4"].pct_change(fill_method=None) * 100

# Preencher NaNs nos retornos com 0
df["Retorno_Dólar_%"] = df["Retorno_Dólar_%"].fillna(0)
df["Retorno_PETR4_%"] = df["Retorno_PETR4_%"].fillna(0)

# Calcular retorno acumulado (começando em 1)
df["Acumulado_Dólar"] = (1 + df["Retorno_Dólar_%"]/100).cumprod()
df["Acumulado_PETR4"] = (1 + df["Retorno_PETR4_%"]/100).cumprod()

# Padronizar formato da data
df["Data"] = pd.to_datetime(df["Data"]).dt.date

# Exportar para Excel
df.to_excel("dados_financeiros.xlsx", index=False)

df.head()

/tmp/ipython-input-3298448102.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2023-01-01", end="2024-12-31")["Close"]
[*********************100%***********************]  2 of 2 completed


,Data,Dólar,PETR4,Retorno_Dólar_%,Retorno_PETR4_%,Acumulado_Dólar,Acumulado_PETR4
0,2023-01-02,13.380746,5.285500,0.000000,0.000000,1.000000,1.000000
1,2023-01-03,13.042145,5.360131,-2.530510,1.411990,0.974695,1.014120
2,2023-01-04,13.456644,5.478500,3.178153,2.208325,1.005672,1.036515
3,2023-01-05,13.941200,5.430100,3.600870,-0.883452,1.041885,1.027358
4,2023-01-06,13.859467,5.349800,-0.586275,-1.478791,1.035777,1.012165


In [ ]:
import yfinance as yf
import pandas as pd

# Lista de ativos brasileiros + dólar + ibov
tickers = ["USDBRL=X", "PETR4.SA", "VALE3.SA", "ITUB4.SA", "BBDC4.SA", "ABEV3.SA", "^BVSP"]

# Baixar dados
data = yf.download(tickers, start="2023-01-01", end="2024-12-31")["Close"]

# Resetar index
df = data.reset_index()
df.rename(columns={"USDBRL=X": "Dólar", "^BVSP": "IBOV", "PETR4.SA": "PETR4", "BBDC4.SA": "BBDC4", "ABEV3.SA": "ABEV3"}, inplace=True)

# Interpolação do dólar (às vezes falha em alguns dias)
df["Dólar"] = df["Dólar"].interpolate()

# Calcular retornos percentuais
retornos = df.set_index("Date").pct_change().fillna(0)

# Calcular acumulados (começando em 1)
acumulados = (1 + retornos).cumprod()

df["Date"] = pd.to_datetime(df["Date"]).dt.date

# Concatenar em um único dataframe
final = pd.concat([retornos.add_suffix("_Retorno"), acumulados.add_suffix("_Acumulado")], axis=1).reset_index()


# Exportar
final.to_excel("dados_financeiros_multiplos3.xlsx", index=False)

print(final.head())


/tmp/ipython-input-3305871395.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2023-01-01", end="2024-12-31")["Close"]
[*********************100%***********************]  7 of 7 completed
/tmp/ipython-input-3305871395.py:18: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos = df.set_index("Date").pct_change().fillna(0)


Ticker       Date  ABEV3_Retorno  BBDC4_Retorno  ITUB4.SA_Retorno  \
0      2023-01-02       0.000000       0.000000          0.000000   
1      2023-01-03      -0.001409      -0.049625         -0.020825   
2      2023-01-04       0.005646       0.002857          0.002919   
3      2023-01-05       0.002807       0.043447          0.031185   
4      2023-01-06       0.002099       0.027987          0.017339   

Ticker  PETR4_Retorno  Dólar_Retorno  VALE3.SA_Retorno  IBOV_Retorno  \
0            0.000000       0.000000          0.000000      0.000000   
1           -0.025305       0.014120         -0.001790     -0.020775   
2            0.031781       0.022083          0.001793      0.011213   
3            0.036009      -0.008835          0.016779      0.020734   
4           -0.005863      -0.014788          0.015842      0.012258   

Ticker  ABEV3_Acumulado  BBDC4_Acumulado  ITUB4.SA_Acumulado  PETR4_Acumulado  \
0              1.000000         1.000000            1.000000         1.

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime

def processar_dados_financeiros():

    # Configuração dos ativos
    tickers = {
        "USDBRL=X": "Dólar",
        "PETR4.SA": "PETR4",
        "VALE3.SA": "VALE3",
        "ITUB4.SA": "ITUB4",
        "BBDC4.SA": "BBDC4",
        "ABEV3.SA": "ABEV3",
        "^BVSP": "IBOV"
    }

    # Baixar dados brutos
    data = yf.download(
        list(tickers.keys()),
        start="2023-01-01",
        end="2024-12-31",
        progress=False
    )["Close"]

    # Renomear colunas
    data.columns = [tickers[col] for col in data.columns]

    # Criar range completo de datas (apenas dias úteis para mercado financeiro)
    date_range = pd.date_range(
        start=data.index.min(),
        end=data.index.max(),
        freq='D'  # Todos os dias
    )

    # Reindexar para incluir todos os dias
    data = data.reindex(date_range)

    # Forward fill para preencher valores nulos com último valor conhecido
    data = data.fillna(method='ffill')

    # Garantir que não há valores nulos no início
    data = data.dropna()

    # Reset index para trabalhar com DataFrame
    df_cotacoes = data.reset_index()
    df_cotacoes.rename(columns={'index': 'Data'}, inplace=True)

    print(f"Dados processados: {len(df_cotacoes)} dias")
    print(f"Período: {df_cotacoes['Data'].min()} até {df_cotacoes['Data'].max()}")

    # Calcular retornos diários (apenas para dias com mudança real de preço)
    df_retornos = df_cotacoes.set_index('Data').pct_change()

    # Zerar retornos em fins de semana/feriados (quando preço não mudou)
    for col in df_retornos.columns:
        # Se o preço não mudou, retorno = 0
        df_retornos[col] = df_retornos[col].where(
            df_cotacoes.set_index('Data')[col].diff() != 0, 0
        )

    # Preencher primeiro dia com 0
    df_retornos = df_retornos.fillna(0)

    # Calcular retornos acumulados
    df_acumulados = (1 + df_retornos).cumprod()

    # Renomear colunas para identificar cada métrica
    df_retornos.columns = [f"{col}_Retorno" for col in df_retornos.columns]
    df_acumulados.columns = [f"{col}_Acumulado" for col in df_acumulados.columns]

    # Combinar todos os DataFrames
    resultado_final = pd.concat([
        df_cotacoes.set_index('Data'),
        df_retornos,
        df_acumulados
    ], axis=1).reset_index()

    # Formatar data
    resultado_final['Data'] = pd.to_datetime(resultado_final['Data']).dt.strftime("%Y-%m-%d")

    # Reordenar colunas
    colunas_ordenadas = ['Data']
    for ticker_nome in tickers.values():
        colunas_ordenadas.extend([
            ticker_nome,
            f"{ticker_nome}_Retorno",
            f"{ticker_nome}_Acumulado"
        ])

    resultado_final = resultado_final[colunas_ordenadas]

    return resultado_final

def exportar_dados(df, nome_arquivo="dados_financeiros_completo.xlsx"):
    """
    Exporta os dados para Excel com formatação
    """
    try:
        df.to_excel(nome_arquivo, index=False)



        # Verificar se há valores nulos
        nulos = df.isnull().sum().sum()
        if nulos == 0:
            print("   Nenhum valor nulo encontrado")
        else:
            print(f"   {nulos} valores nulos encontrados")

    except Exception as e:
        print(f"❌ Erro ao exportar: {e}")

def main():

    try:
        # Processar dados
        dados_finais = processar_dados_financeiros()
        # Exportar
        exportar_dados(dados_finais)
        dados_finais.to_excel("dados_financeiros_multiplos_corrigido.xlsx", index=False)

        return dados_finais

    except Exception as e:
        print(f"❌ Erro no pipeline: {e}")
        return None

# Executar pipeline
if __name__ == "__main__":
    df_final = main()

/tmp/ipython-input-3511745945.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
/tmp/ipython-input-3511745945.py:40: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


Dados processados: 729 dias
Período: 2023-01-02 00:00:00 até 2024-12-30 00:00:00
   • ✅ Nenhum valor nulo encontrado


In [7]:
import numpy as np

def calcular_metricas(df, tickers):
    """
    Calcula retorno anualizado, volatilidade anualizada e Sharpe Ratio
    para cada ativo.
    """
    resultados = []

    # Loop em cada ativo da sua lista
    for ativo in tickers.values():
        col_retorno = f"{ativo}_Retorno"

        # Retornos diários
        retornos = df[col_retorno]

        # Retorno médio diário
        retorno_medio = retornos.mean()

        # Retorno anualizado
        retorno_anual = (1 + retorno_medio) ** 252 - 1

        # Volatilidade anualizada
        volatilidade = retornos.std() * np.sqrt(252)

        # Sharpe Ratio (assumindo taxa livre de risco = 0)
        sharpe = retorno_anual / volatilidade if volatilidade != 0 else 0

        resultados.append({
            "Ativo": ativo,
            "Retorno_Anualizado_%": round(retorno_anual, 4),
            "Volatilidade_%": round(volatilidade, 4),
            "Sharpe": round(sharpe, 2)
        })

    return pd.DataFrame(resultados)


if __name__ == "__main__":
    df_final = main()

    if df_final is not None:

        tickers = {
            "USDBRL=X": "Dólar",
            "PETR4.SA": "PETR4",
            "VALE3.SA": "VALE3",
            "ITUB4.SA": "ITUB4",
            "BBDC4.SA": "BBDC4",
            "ABEV3.SA": "ABEV3",
            "^BVSP": "IBOV"
        }

        resumo = calcular_metricas(df_final, tickers)
        resumo.to_excel("metricas_resumo.xlsx", index=False)


/tmp/ipython-input-3427930876.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
/tmp/ipython-input-3427930876.py:40: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


Dados processados: 729 dias
Período: 2023-01-02 00:00:00 até 2024-12-30 00:00:00

📋 Primeiros 10 registros:
      Data    Dólar  Dólar_Retorno  Dólar_Acumulado     PETR4  PETR4_Retorno  PETR4_Acumulado     VALE3  VALE3_Retorno  VALE3_Acumulado     ITUB4  ITUB4_Retorno  ITUB4_Acumulado     BBDC4  BBDC4_Retorno  BBDC4_Acumulado     ABEV3  ABEV3_Retorno  ABEV3_Acumulado     IBOV  IBOV_Retorno  IBOV_Acumulado
2023-01-02 5.285500       0.000000         1.000000 13.380747       0.000000         1.000000 70.104393       0.000000         1.000000 18.484621       0.000000         1.000000 11.706137       0.000000         1.000000 12.548673       0.000000         1.000000 106376.0      0.000000        1.000000
2023-01-03 5.360131       0.014120         1.014120 13.042144      -0.025305         0.974695 69.978935      -0.001790         0.998210 18.099670      -0.020825         0.979175 11.125226      -0.049624         0.950376 12.530987      -0.001409         0.998591 104166.0     -0.020775      